In [1]:
import os
import requests
import subprocess

def download_geofabrik_pbf(years, base_url="https://download.geofabrik.de/europe/"):
    folder = "osm_geofabrik_pbf"
    os.makedirs(folder, exist_ok=True)
    
    for year in years:
        filename = f"germany-{year}0101.osm.pbf"
        file_path = os.path.join(folder, filename)
        file_url = base_url + filename
        
        if os.path.exists(file_path):
            print(f"File already exists: {file_path}, skipping download.")
        else:
            print(f"Downloading: {file_url}")
            response = requests.get(file_url, stream=True)
            if response.status_code == 200:
                with open(file_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        f.write(chunk)
                print(f"Downloaded: {file_path}")
            else:
                print(f"Failed to download {file_url} (Status code: {response.status_code})")

def run_osmium(year):
    try:
        folder = "processed_osm_files"
        os.makedirs(folder, exist_ok=True)
        
        input_pbf = os.path.join("osm_geofabrik_pbf", f"germany-{year}0101.osm.pbf")
        filtered_pbf = os.path.join(folder, f"processed_roads_germany_{year}.pbf")

        filter_command = [
            "osmium", "tags-filter",
            input_pbf,
            "w/highway=primary,primary_link,secondary,secondary_link,tertiary,tertiary_link",
            "-o", filtered_pbf
        ]
        print("🔹 Running: ", " ".join(filter_command))
        subprocess.run(filter_command, check=True)

        print("✅ Osmium processing complete! Files saved in 'processed_osm_files/'")

    except subprocess.CalledProcessError as e:
        print("❌ Error running Osmium:", e)

if __name__ == "__main__":
    years = [18, 24]  # Configure the years you need
    download_geofabrik_pbf(years)
    
    for year in years:
        run_osmium(str(year))


Downloading: https://download.geofabrik.de/europe/germany-180101.osm.pbf
Downloaded: osm_geofabrik_pbf\germany-180101.osm.pbf
Downloading: https://download.geofabrik.de/europe/germany-240101.osm.pbf
Downloaded: osm_geofabrik_pbf\germany-240101.osm.pbf
🔹 Running:  osmium tags-filter osm_geofabrik_pbf\germany-180101.osm.pbf w/highway=primary,primary_link,secondary,secondary_link,tertiary,tertiary_link -o processed_osm_files\processed_roads_germany_18.pbf
✅ Osmium processing complete! Files saved in 'processed_osm_files/'
🔹 Running:  osmium tags-filter osm_geofabrik_pbf\germany-240101.osm.pbf w/highway=primary,primary_link,secondary,secondary_link,tertiary,tertiary_link -o processed_osm_files\processed_roads_germany_24.pbf
✅ Osmium processing complete! Files saved in 'processed_osm_files/'
